# Python Notebook for calculations likelihood root-based attacks against maximal real extensions of the forn $n = 2^r \cdot p^s$

# Dependencies

In [ ]:
# Initial dependencies to be used
!pip install galois, sympy, joblib, math, numpy

In [38]:
# work with number fields, primes
import galois, sympy
# parallelized executions
from joblib import Parallel, delayed
# useful math functions for root-based calculations
from math import erf, ceil, sqrt, floor
# Chebychev factorization
import numpy as np

# Initial Definitions and Auxiliary Processes

To analyze the case of roots over small finite extensions of $\mathbb{F}_q$, and in an attempt to bring down the computational cost of finding roots, we will in this section get the factors of the maximal real polynomial

In [60]:
#Chebychev for the sum of T_i(x) terms
cheb = np.polynomial.chebyshev.Chebyshev([1,1,1])
print(cheb)

1.0 + 1.0 T_1(x) + 1.0 T_2(x)


The idea is, first get the terms in which a $V_i(x) = 2 \cdot T_i(x/2)$ is. Then, calculate the Chebychev coefficients there. And, lastly, interate over each to add the term $\frac{1}{2}^i$.

Now, we define the minimal polynomials of the maximal real extensions in the cases $n = p^s$ and $n = 2^r \cdot p^s$

In [95]:
def minimal_polynomial_case_ps(p, s, q):
    F = galois.GF(q)
    k = (p-1)//2
    term = (p**(s-1))
    terms = [0 for x in range(0, k*term + 1)]
    terms[0] = 1
    for i in range(1, k+1):
        coeff = i*term
        terms[coeff] = 2
    
    #Get the Chebychev coeffs
    cheb = np.polynomial.chebyshev.Chebyshev(terms)
    coef = [F(int(x % q)) for x in np.polynomial.chebyshev.cheb2poly(cheb.coef)]
    coeffs = []
    for i in range(len(coef)):
        coeffs.append(coef[i] / (F(2)**i))
    return galois.Poly(coeffs[::-1], field=F)

def minimal_polynomial_case_psr(p, s, r, q):
    #Case r = 0 represents n = p^s
    if r == 0:
        return minimal_polynomial_case_ps(p, s, q)
    F = galois.GF(q)
    k = (p-1)//2
    term = (p**(s-1)) * (2**(r-1))
    terms = [0 for x in range(0, k*term + 1)]
    terms[0] = 1*((-1)**k)
    for i in range(1, k+1):
        coeff = i*term
        terms[coeff] = 2*((-1)**(k-i))
        
    #Get the Chebychev coeffs
    cheb = np.polynomial.chebyshev.Chebyshev(terms)
    coef = [F(int(x % q)) for x in np.polynomial.chebyshev.cheb2poly(cheb.coef)]
    coeffs = []
    for i in range(len(coef)):
        coeffs.append(coef[i] / (F(2)**i))
    return galois.Poly(coeffs[::-1], field=F)

In [103]:
p = 5
s = 1
r = 3
q = 7

x = 1

pol = minimal_polynomial_case_psr(p, s, r, q)
print(f'The result of the minimal polynomial {pol} at x = {x} for p = {p}, s = {s} and r = {r} in GF({q}) is {pol(x)}')

r = 0
pol = minimal_polynomial_case_psr(p, s, r, q)
print(f'The result of the minimal polynomial {pol} at x = {x} for p = {p}, s = {s} and r = {r} in GF({q}) is {pol(x)}')

The result of the minimal polynomial x^8 + 6x^6 + 5x^4 + 2x^2 + 1 at x = 1 for p = 5, s = 1 and r = 3 in GF(7) is 1
The result of the minimal polynomial x^2 + x + 6 at x = 1 for p = 5, s = 1 and r = 0 in GF(7) is 1


In [ ]:
for x in pol.roots():
    print(f'Root x = {int(x)} with multiplicative order r = {x.multiplicative_order()}')

In [ ]:
factors, multiplicities = pol.factors()

# Loop through the factors and their corresponding multiplicities
for factor, multiplicity in zip(factors, multiplicities):
    print(f"Factor {factor} of degree {factor.degree} with multiplicity {multiplicity}")

Now, we define the calculations for each of the root-based attacks that will be studied (over the base field $\mathbb{F}_q$ and finite small extensions of it):

* Attack on the Small Set of Error Values
* Attack on the Small Error Values
* Attack on the Small Error Values (Fully Probabilistic, Generalized)

In [ ]:
#Small Set of Error Values Attack ratio

def likelihood_small_set(sigma, n, r, q):
    block = ((4*n*sigma) / r)
    # provision to handle 'extremely big' numbers
    if (block >= 1.4 and r >= 128) or n < r:
        return 1
    card_set = block**r
    return card_set/q

In [ ]:
# Get Small Set of Error Values change of each instance

def process_outer_task_small_set_attack_chances(p, s, r, q, sigma, order):
    n = (2**(r-2))*(p**(s-1))*(p-1)
    return likelihood_small_set(sigma, n, order, q) < 1

# Main parallelized execution
def parallelized_task_small_set_attack_chances(list_total_info, sigma):
    list_chances_ssa = []
    results = Parallel(n_jobs=-1)(delayed(process_outer_task_small_set_attack_chances)(p, s, r, q, sigma, order)
                                  for (p, s, r, q, alpha, order) in list_total_info)
    for result in results:
        list_chances_ssa.append(result)
    return list_chances_ssa

def process_outer_task_small_set_attack_chances_k_ideals(p, s, r, q, sigma, order, k):
    if order == float('inf'):
        #not a real k-ideal factor
        return False
    # Account for k-division due to the trace
    n = (2**(r-2))*(p**(s-1))*(p-1)//k
    return likelihood_small_set(sigma, n, order, q) < 1

# Main parallelized execution
def parallelized_task_small_set_attack_chances_k_ideals(list_total_info, sigma):
    list_chances_ssa = []
    results = Parallel(n_jobs=-1)(delayed(process_outer_task_small_set_attack_chances_k_ideals)(p, s, r, q, sigma, order, k)
                                  for (p, s, r, q, value, order, k) in list_total_info)
    for result in results:
        list_chances_ssa.append(result)
    return list_chances_ssa

def small_err_set_attack_ratio(list_chances_ssa):
    return len([p for p in list_chances_ssa if p == True])/len(list_chances_ssa)

In [ ]:
#Small Error Values Attack (with Fully Probabilistic Approach)

#Definitions of each distribution case
def case_distribution_1(n, sigma):
  quot = 1
  factor = (sigma**2)*n
  return sqrt(factor*quot)

def case_distribution_2(n, sigma, root, ord):
  quot = ((root**(2*ord)) - 1)//((root**2) - 1)
  factor = (n/ord)*(sigma**2)
  return sqrt(factor*quot)

def case_distribution_3(n, sigma, root):
  quot = ((root**(2*n)) - 1)//((root**2) - 1)
  factor = sigma**2
  return decimal.Decimal(factor*quot).sqrt()

# Auxiliary function to compute the integral that reigns over the success probability of the fully probabilistic approach
def A(r):
  return (erf(r/4))/2

def N(r):
  return (sqrt(2)/r) - (1/2)

def Bterm(k, r):
  return (erf(r*(k + (5/4))) - erf(r*(k + (3/4))))/2

def B(r):
  bound = N(r)
  value = 0.0
  for k in range(floor(bound)):
      value += Bterm(k , r)
  if bound > 0:
    correction_bound = bound - floor(bound)
    if correction_bound <= 1/4:
      correction_rate = 0
    elif correction_bound < 3/4:
      correction_rate = (erf(sqrt(2)) - erf(r*(floor(bound) + (3/4))))/2
    else:
      correction_rate = (erf(r*(floor(bound) + (5/4))) - erf(r*(floor(bound) + (3/4))))/2
    value += correction_rate
  return value

def I(r):
  return (A(r) + B(r))/(erf(sqrt(2))/2)

def Icota(r):
  return I(r) - 1/2

In [ ]:
# Get Small Error Values change of each instance
def likelihood_small_err_values_chances(p, s, r, q, sigma, alpha, order):
    n = (2**(r-2))*(p**(s-1))*(p-1)
    if order == 1:
        sigma_image = case_distribution_1(n, sigma)
    elif order < 5:
        sigma_image = case_distribution_2(n, sigma, alpha, order)
    else:
        sigma_image = float(case_distribution_3(n, sigma, alpha))
    if (q/4 < 2*sigma_image):
        ratio = q/(sqrt(2) * sigma_image)
        if ratio == 0:
            return 0
        else:
            return Icota(ratio)
    else:
        return 1

# Main parallelized execution
def parallelized_task_small_err_values_chances(list_total_info, sigma):
    list_chances_sva = []
    results = Parallel(n_jobs=-1)(delayed(likelihood_small_err_values_chances)(p, s, r, q, sigma, alpha, order)
                                  for (p, s, r, q, alpha, order) in list_total_info)
    for result in results:
        list_chances_sva.append(result)
    return list_chances_sva

def likelihood_small_err_values_chances_k_ideals(p, s, r, q, sigma, value, order, k):
    if value == None:
        # not a real k-ideal factor
        return 0
    n = (2**(r-2))*(p**(s-1))*(p-1)//k
    if n < order:
        # n is supposed to be divisible by r (at least smaller), otherwise the computation does not make sense
        return 0
    if order == 1:
        sigma_image = case_distribution_1(n, sigma)
    elif order < 5:
        sigma_image = case_distribution_2(n, sigma, alpha, order)
    else:
        sigma_image = float(case_distribution_3(n, sigma, alpha))
    if (q/4 < 2*sigma_image):
        ratio = q/(sqrt(2) * sigma_image)
        if ratio == 0:
            return 0
        else:
            return Icota(ratio)
    else:
        return 1

# Main parallelized execution
def parallelized_task_small_err_values_chances_k_ideals(list_total_info, sigma):
    list_chances_sva = []
    results = Parallel(n_jobs=-1)(delayed(likelihood_small_err_values_chances_k_ideals)(p, s, r, q, sigma, value, order, k)
                                  for (p, s, r, q, value, order, k) in list_total_info)
    for result in results:
        list_chances_sva.append(result)
    return list_chances_sva

def small_err_value_ratio(list_chances_sva):
    return len([p for p in list_chances_sva if p > 0])/len(list_chances_sva)

# Techniques for study over $\mathbb{F}_q$ (i.e, using roots directly)

Next, we introduce the processes we will run in order to get the desired data:

* Number of instances with roots of small order (i.e. order < 5)
* Root of minimal order of each instance

In [ ]:
# Number of small order roots in a certain instance

def process_outer_task_small_roots(p, s, r, q):
    list_info = []

    pol = minimal_polynomial_case_psr(p, s, r, q)
    for x in pol.roots():
        order = x.multiplicative_order()
        if order < 5:
            list_info.append((p, s, r, q, int(x), order))
    return list_info

def parallelized_task_small_roots(list_triples, list_q_values):
    list_total_info = []
    results = Parallel(n_jobs=-1)(delayed(process_outer_task_small_roots)(p, s, r, q)
                                  for (p, s, r) in list_triples
                                  for q in list_q_values)
    for result in results:
        list_total_info.extend(result)
    return list_total_info

In [ ]:
# Smallest-order root of each instance

def process_outer_task_smallest_root(p, s, r, q):
    list_info = []
    min_order = float('inf')
    best_root = None
    
    pol = minimal_polynomial_case_psr(p, s, r, q)
    for x in pol.roots():
        order = x.multiplicative_order()
        if order < min_order:
            best_root = int(x)
            min_order = order
    list_info.append((p, s, r, q, best_root, min_order))
    return list_info

# Main parallelized execution
def parallelized_task_smallest_root(list_triples, list_q_values):
    list_total_info = []
    results = Parallel(n_jobs=-1)(delayed(process_outer_task_smallest_root)(p, s, r, q)
                                  for (p, s, r) in list_triples
                                  for q in list_q_values)
    for result in results:
        list_total_info.extend(result)
    return list_total_info

# Techinques for study over finite extensions of $\mathbb{F}_q$ (of degree up to $k = 4$)

Here, we won't be interested in roots of the polynomials but, based upon the works of Blanco et al., on $k$-ideal factors, i.e. factors of the form $x^k + a$, were $a \in \mathbb{F}_q$ and of small order

Next, we introduce the processes we will run in order to get the desired data:

* Number of instances with k-ideal factors with independent element of small order (i.e. order < 5)
* k-ideal factor of minimal order of each instance

In [114]:
# Number of k-ideal factors in a certain instance

def process_outer_task_small_k_ideal_factors(p, s, r, q):
    list_info = []

    pol = minimal_polynomial_case_psr(p, s, r, q)
    factors, multiplicities = pol.factors()
    for g in factors:
        degree = g.degree
        for k in range(2, 5):
            if degree == k:
                non_zero_degrees = [int(x) for x in g.nonzero_degrees]
                if len(non_zero_degrees) == 2 and non_zero_degrees[1] == 0:
                    non_zero_coeffs = [x for x in g.nonzero_coeffs]
                    order_independent_term = non_zero_coeffs[1].multiplicative_order()
                    if order_independent_term < 5:
                        list_info.append((p, s, r, q, int(non_zero_coeffs[1]), order_independent_term, k))
    return list_info

def parallelized_task_small_k_ideal_factors(list_triples, list_q_values):
    list_total_info = []
    results = Parallel(n_jobs=-1)(delayed(process_outer_task_small_k_ideal_factors)(p, s, r, q)
                                  for (p, s, r) in list_triples
                                  for q in list_q_values)
    for result in results:
        list_total_info.extend(result)
    return list_total_info

In [115]:
# Number of k-ideal factors in a certain instance

def process_outer_task_smallest_k_ideal_factors(p, s, r, q):
    list_info = []

    pol = minimal_polynomial_case_psr(p, s, r, q)
    factors, multiplicities = pol.factors()
    for k in range(2, 5):
        min_order = float('inf')
        best_term = None
        for g in factors:
            if g.degree == k:
                non_zero_degrees = [int(x) for x in g.nonzero_degrees]
                if len(non_zero_degrees) == 2 and non_zero_degrees[1] == 0:
                    non_zero_coeffs = [x for x in g.nonzero_coeffs]
                    order_independent_term = (-non_zero_coeffs[1]).multiplicative_order()
                    if order_independent_term < min_order:
                        min_order = order_independent_term
                        best_term = int(non_zero_coeffs[1])
                        
        list_info.append((p, s, r, q, best_term, min_order, k))
    return list_info

def parallelized_task_smallest_k_ideal_factors(list_triples, list_q_values):
    list_total_info = []
    results = Parallel(n_jobs=-1)(delayed(process_outer_task_smallest_k_ideal_factors)(p, s, r, q)
                                  for (p, s, r) in list_triples
                                  for q in list_q_values)
    for result in results:
        list_total_info.extend(result)
    return list_total_info

# Study over standardized setting of structured lattices (ML-KEM, ML-DSA and FN-DSA)

In [116]:
list_q_values = [3329, 12289, 8380417]
list_n_values = [256, 512, 1024]

First, we have to select some values of (p, s, r) which are closest to the values of $n$ to study (with the additional restriction of surpassing $n$).

In [ ]:
def closest_factors(n):
    primes = list(sympy.primerange(5, 2*n))
    
    best_r = 0
    best_p = 0
    best_s = 0
    best_diff = float('inf')
    
    for p in primes:
        s = 1
        while True:
            power_of_p = (p ** (s-1)) * ((p-1)//2) 
            if power_of_p - n > best_diff:
                break
    
            r = 2
            while 2 ** (r-1) * power_of_p - n <= best_diff:
                diff = 2 ** (r-1) * power_of_p - n
                if diff < best_diff and diff >= 0:
                    best_r, best_p, best_s = r, p, s
                    best_diff = diff
                r += 1
            s += 1
    
    return best_r, best_p, best_s

def closest_factors_no_r(n):
    primes = list(sympy.primerange(5, n + 1))
    
    best_r = 0
    best_p = 0
    best_s = 0
    best_diff = float('inf')
    
    for p in primes:
        s = 1
        while True:
            power_of_p = (p ** (s-1)) * ((p-1)//2) 
            if power_of_p - n > best_diff:
                break
    
            diff = power_of_p - n
            if diff < best_diff and diff >= 0:
                best_p, best_s = p, s
                best_diff = diff
            s += 1
    
    return best_r, best_p, best_s

In [ ]:
# Example usage:
n = 256
r, p, s = closest_factors(n)
print(f"For n = {n}, the closest r, p, s are: r = {r}, p = {p}, s = {s}")
r, p, s = closest_factors_no_r(n)
print(f"For n = {n}, the closest p, s are: p = {p}, s = {s}")

In [ ]:
list_instances_n_setting = []
for n in list_n_values:
    r, p, s = closest_factors(n)
    list_instances_n_setting.append((p, s, r))
list_instances_n_setting

ML-KEM setting (n = 256, q = 3329)

In [117]:
pol = minimal_polynomial_case_psr(5, 1, 8, 3329)
pol.factors()

([Poly(x + 1134, GF(3329)),
  Poly(x + 1385, GF(3329)),
  Poly(x + 1944, GF(3329)),
  Poly(x + 2195, GF(3329)),
  Poly(x^2 + 1055, GF(3329)),
  Poly(x^28 + 2385x^26 + 1744x^24 + 2387x^22 + 328x^20 + 2901x^18 + x^16 + 2216x^14 + 2830x^12 + 764x^10 + 50x^8 + 1908x^6 + 341x^4 + 1503x^2 + 1229, GF(3329)),
  Poly(x^28 + 675x^27 + 3315x^26 + 1504x^25 + 2629x^24 + 215x^23 + 1205x^22 + 245x^21 + 1716x^20 + 1376x^19 + 2780x^18 + 2998x^17 + 106x^16 + 1541x^15 + 2140x^14 + 695x^13 + 2757x^12 + 3316x^11 + 2306x^10 + 1367x^9 + 735x^8 + 1318x^7 + 686x^6 + 1423x^5 + 712x^4 + 2399x^3 + 1213x^2 + 1483x + 1444, GF(3329)),
  Poly(x^28 + 2654x^27 + 3315x^26 + 1825x^25 + 2629x^24 + 3114x^23 + 1205x^22 + 3084x^21 + 1716x^20 + 1953x^19 + 2780x^18 + 331x^17 + 106x^16 + 1788x^15 + 2140x^14 + 2634x^13 + 2757x^12 + 13x^11 + 2306x^10 + 1962x^9 + 735x^8 + 2011x^7 + 686x^6 + 1906x^5 + 712x^4 + 930x^3 + 1213x^2 + 1846x + 1444, GF(3329)),
  Poly(x^83 + 648x^82 + 673x^81 + 1830x^80 + 1954x^79 + 1065x^78 + 2104x^77 + 1

In [118]:
list_info_small_roots_ml_kem = parallelized_task_small_roots([(5, 1, 8)], [3329])
list_info_small_roots_ml_kem

[]

In [119]:
list_info_smallest_roots_ml_kem = parallelized_task_smallest_root([(5, 1, 8)], [3329])
list_info_smallest_roots_ml_kem

[(5, 1, 8, 3329, 1134, 832)]

Attack analysis over finite extensions of $\mathbb{F}_q$

In [ ]:
list_info_small_k_ideal_factors_ml_kem = parallelized_task_small_k_ideal_factors([(5, 1, 8)], [3329])
list_info_small_k_ideal_factors_ml_kem

In [ ]:
list_info_smallest_k_ideal_factors_ml_kem = parallelized_task_smallest_k_ideal_factors([(5, 1, 8)], [3329])
list_info_smallest_k_ideal_factors_ml_kem

In [ ]:
# Small Set Error rate on Standardized setting
sigma = 2
small_err_set_chances_ml_kem = parallelized_task_small_set_attack_chances(list_info_smallest_roots_ml_kem, sigma)
small_err_set_attack_ratio(small_err_set_chances_ml_kem)

In [ ]:
# Small Set Error rate on Standardized setting
sigma = 2
small_err_set_chances_ml_kem_k_ideals = parallelized_task_small_set_attack_chances_k_ideals(list_info_smallest_k_ideal_factors_ml_kem, sigma)
small_err_set_attack_ratio(small_err_set_chances_ml_kem_k_ideals)

In [ ]:
# Small Error Values rate on Standardized setting
sigma = 2
small_err_values_chances_ml_kem = parallelized_task_small_err_values_chances(list_info_smallest_roots_ml_kem, sigma)
small_err_value_ratio(small_err_values_chances_ml_kem)

In [ ]:
# Small Error Values rate on Standardized setting
sigma = 2
small_err_values_chances_ml_kem_k_ideals = parallelized_task_small_err_values_chances_k_ideals(list_info_smallest_k_ideal_factors_ml_kem, sigma)
small_err_value_ratio(small_err_values_chances_ml_kem_k_ideals)

Result: No applicable attack against both roots and finite field extensions for "ML-KEM-like" setting

ML-DSA setting (n = 256, q = 8380417)

In [ ]:
pol = minimal_polynomial_case_psr(5, 1, 8, 8380417)
pol.factors()

In [ ]:
list_info_small_roots_ml_dsa = parallelized_task_small_roots([(5, 1, 8)], [8380417])
list_info_small_roots_ml_dsa

In [ ]:
list_info_smallest_roots_ml_dsa = parallelized_task_smallest_root([(5, 1, 8)], [8380417])
list_info_smallest_roots_ml_dsa

In [ ]:
list_info_small_k_ideal_factors_ml_dsa = parallelized_task_small_k_ideal_factors([(5, 1, 8)], [8380417])
list_info_small_k_ideal_factors_ml_dsa

In [ ]:
list_info_smallest_k_ideal_factors_ml_dsa = parallelized_task_smallest_k_ideal_factors([(5, 1, 8)], [8380417])
list_info_smallest_k_ideal_factors_ml_dsa

In [ ]:
# Small Set Error rate on Standardized setting
sigma = 2
small_err_set_chances_ml_dsa_k_ideals = parallelized_task_small_set_attack_chances_k_ideals(list_info_smallest_k_ideal_factors_ml_dsa, sigma)
small_err_set_attack_ratio(small_err_set_chances_ml_dsa_k_ideals)

In [ ]:
# Small Error Values rate on Standardized setting
sigma = 2
small_err_values_chances_ml_dsa_k_ideals = parallelized_task_small_err_values_chances_k_ideals(list_info_smallest_k_ideal_factors_ml_dsa, sigma)
small_err_value_ratio(small_err_values_chances_ml_dsa_k_ideals)

Result: No applicable attack against both roots and finite field extensions for "ML-DSA-like" setting

FN-DSA setting (n = 512, 1024, q = 12289)

In [ ]:
pol = minimal_polynomial_case_psr(5, 1, 9, 12289)
pol.factors()

In [ ]:
list_info_small_roots_fn_dsa = parallelized_task_small_roots([(5, 1, 9)], [12289])
list_info_small_roots_fn_dsa

In [ ]:
list_info_smallest_roots_fn_dsa = parallelized_task_smallest_root([(5, 1, 9)], [12289])
list_info_smallest_roots_fn_dsa

In [ ]:
list_info_small_k_ideal_factors_fn_dsa = parallelized_task_small_k_ideal_factors([(5, 1, 9)], [12289])
list_info_small_k_ideal_factors_fn_dsa

In [ ]:
list_info_smallest_k_ideal_factors_fn_dsa = parallelized_task_smallest_k_ideal_factors([(5, 1, 9)], [12289])
list_info_smallest_k_ideal_factors_fn_dsa

In [ ]:
# Small Set Error rate on Standardized setting
sigma = 2
small_err_set_chances_fn_dsa = parallelized_task_small_set_attack_chances(list_info_smallest_roots_fn_dsa, sigma)
small_err_set_attack_ratio(small_err_set_chances_fn_dsa)

In [ ]:
# Small Error Values rate on Standardized setting
sigma = 2
small_err_values_chances_fn_dsa = parallelized_task_small_err_values_chances(list_info_smallest_roots_fn_dsa, sigma)
small_err_value_ratio(small_err_values_chances_fn_dsa)

Result: No applicable attack against both roots and finite field extensions for "FN-DSA-like" setting

# General study of random instances ($n \in [128, 256]$, $q \in (1024, 8192)$)

Now we analyze how random instances, in general, can be affected by these kind of attacks.

In [89]:
bottom_n = 128
top_n = 256

bottom_q = 1024
top_q = 4096

bottom_sigma = 2
top_sigma = 8

In [90]:
list_p_values = list(sympy.primerange(5, 100))
list_s_values = [1]
list_r_values = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

list_instances_n_setting = []
for r in list_r_values:
  for s in list_s_values:
    for p in list_p_values:
      n = (p**(s-1))*(2**(r-2))*(p-1)
      if bottom_n <= n <= top_n:
        list_instances_n_setting.append((p, s, r))
print(f'We get {len(list_instances_n_setting)} combinations which yield a polynomial of degree in [{bottom_n}, {top_n}]')

We get 25 combinations which yield a polynomial of degree in [128, 256]


In [91]:
list_instances_n_setting

[(67, 1, 3),
 (71, 1, 3),
 (73, 1, 3),
 (79, 1, 3),
 (83, 1, 3),
 (89, 1, 3),
 (97, 1, 3),
 (37, 1, 4),
 (41, 1, 4),
 (43, 1, 4),
 (47, 1, 4),
 (53, 1, 4),
 (59, 1, 4),
 (61, 1, 4),
 (17, 1, 5),
 (19, 1, 5),
 (23, 1, 5),
 (29, 1, 5),
 (31, 1, 5),
 (11, 1, 6),
 (13, 1, 6),
 (17, 1, 6),
 (5, 1, 7),
 (7, 1, 7),
 (5, 1, 8)]

In [92]:
list_q_values = list(sympy.primerange(bottom_q, top_q))
print(f'We get {len(list_q_values)} prime modulus of at most 13 bits, and minimum 10 bits')

We get 392 prime modulus of at most 13 bits, and minimum 10 bits


In [ ]:
list_info_small_roots_generic = parallelized_task_small_roots(list_instances_n_setting, list_q_values)
list_info_small_roots_generic

In [ ]:
list_info_smallest_roots_generic = parallelized_task_smallest_root(list_instances_n_setting, list_q_values)
list_info_smallest_roots_generic

In [93]:
list_sigma_values = list(sympy.primerange(bottom_sigma, top_sigma))
print(f'We get {len(list_sigma_values)} standard deviation values for the associated discrete Gaussian distribution')

We get 4 standard deviation values for the associated discrete Gaussian distribution


In [ ]:
list_sigma_ssa = []
for sigma in list_sigma_values:
    small_err_set_chances = parallelized_task_small_set_attack_chances(list_info_smallest_roots_generic, sigma)
    list_sigma_ssa.append(small_err_set_attack_ratio(small_err_set_chances))
list_sigma_ssa

In [ ]:
list_sigma_sva = []
for sigma in list_sigma_values:
    small_err_values_chances = parallelized_task_small_err_values_chances(list_info_smallest_roots_generic, sigma)
    list_sigma_sva.append(small_err_value_ratio(small_err_values_chances))
list_sigma_sva